## 最も人気なバトルロワイヤルゲームであるPUBGへの探索的データ解析（EDA）

In [ ]:
#この記事では、上の 'EDA is Fun' という記事を日本語訳していきます。著者であるDimitrios Effrosynidisには経緯と感謝を表します。
#https://www.kaggle.com/deffro/eda-is-fun

<img src="https://pmcvariety.files.wordpress.com/2018/04/pubg.jpg?w=1000&h=563&crop=1" alt="PUBG" style="width: 750px;"/>

このカーネルで分析していく特徴量は以下の通り。
- [The Killers](#The-Killers)
- [The Runners](#The-Runners)
- [The Drivers](#The-Drivers)
- [The Swimmers](#The-Swimmers)
- [The Healers](#The-Healers)
- [Solos, Duos and Squads](#Solos,-Duos-and-Squads)
- [Correlation](#Pearson-correlation-between-variables)
- [Feature Engineering](#Feature-Engineering)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")

In [ ]:
pwd

In [ ]:
ls

In [ ]:
cd ../

In [ ]:
cd kaggle

In [ ]:
ls

In [ ]:
cd input/

In [ ]:
ls

In [ ]:
train = pd.read_csv('../input/train_V2.csv')

In [ ]:
train.info()

- **groupId** - 対決におけるグループIDで整数型。もし、同じグループのプレイヤーが別の対決でプレイした場合、その度に別のgroupIDが付与されます。
- **matchId** - 対決ごとに付与される整数型のID。訓練データとテストデータには、同じ対決IDは存在しません。
- **assists** - チームメイトがアシストして、敵を倒した数。
- **boosts** - ブーストアイテムが使われた数。
- **damageDealt** - 受けたダメージ。ノート：自爆は除く。
- **DBNOs** - 攻撃した敵の人数。
- **headshotKills** - ヘッドショットで倒した敵の人数。
- **heals** - 癒しアイテムを使用した回数。
- **killPlace** - その対決において倒した敵の人数に関する順位。
- **killPoints** - 敵を倒した数に基づいたプレイヤー順位。（これについては、倒した数に関するイロレーティングだと考えてください。）
- **kills** -  倒した敵の人数。
- **killStreaks** - 敵のプレイヤーが短い時間内で倒された最大の数。
- **longestKill** - 倒したプレイヤーと倒されたプレイヤーの距離の最大値。 これはミスリーディングの可能性があります。なぜならプレイヤーとドライバーは、これに当てはまる可能性があるからです。
- **maxPlace** - その対戦における最悪の順位。
- **numGroups** - チーム数。
- **revives** - チームメイトを復活させた回数。
- **rideDistance** - 車で移動した距離の合計。
- **roadKills** - 車に乗っている間に倒した敵の人数。
- **swimDistance** - 泳いで移動した距離の合計。
- **teamKills** - チームメイトを倒した回数。
- **vehicleDestroys** - 車が破壊された数。
- **walkDistance** - 歩いて移動した距離。
- **weaponsAcquired** - 拾った武器の数。
- **winPoints** -  勝利ポイントに関するプレイヤー順位。（これについては、勝利ポイントに対するイロレーティングだと思ってください。）
- **winPlacePerc** - 予測のターゲットになります。これは順位についての百分位数になります。順位の割合(1: 1位, 0: 最下位)

In [ ]:
train.head()

オーケー、まずは一目データをみてみましょう。そして、探索の始まりです！

## The Killers

<img src="https://i.ytimg.com/vi/rnAeX795Jn0/maxresdefault.jpg" alt="The Killers" style="width: 700px;"/>

In [ ]:
print("The average person kills {:.4f} players, 99% of people have {} kills or less, while the most kills ever recorded is {}.".format(train['kills'].mean(),train['kills'].quantile(0.99), train['kills'].max()))

まずは倒した数という観点からやっていきましょう。

In [ ]:
data = train.copy()
data.loc[data['kills'] > data['kills'].quantile(0.99)] = '8+'
plt.figure(figsize=(15,10))
sns.countplot(data['kills'].astype('str').sort_values())
plt.title("Kill Count",fontsize=15)
plt.show()

多くの人は1人も倒していないみたいですね。少なくともダメージは与えたのでしょうか？

In [ ]:
data = train.copy()
data = data[data['kills']==0]
plt.figure(figsize=(15,10))
plt.title("Damage Dealt by 0 killers",fontsize=15)
sns.distplot(data['damageDealt'])
plt.show()

おや、ほとんどの人はダメージすら与えていないみたいですね。例外について調べていきましょう。

In [ ]:
print("{} players ({:.4f}%) have won without a single kill!".format(len(data[data['winPlacePerc']==1]), 100*len(data[data['winPlacePerc']==1])/len(train)))

data1 = train[train['damageDealt'] == 0].copy()
print("{} players ({:.4f}%) have won without dealing damage!".format(len(data1[data1['winPlacePerc']==1]), 100*len(data1[data1['winPlacePerc']==1])/len(train)))

順位と倒した数に何かの関連性があるか調べてみましょう。

In [ ]:
sns.jointplot(x="winPlacePerc", y="kills", data=train, height=10, ratio=3, color="r")
plt.show()

どうやら、倒した数と順位には相関があるみたいですね。最終的に「倒した数」でビン分割して、グループ分けしてみましょう。(0人, 1-2 人, 3-5人, 6-10人, 10名以上)

In [ ]:
kills = train.copy()

kills['killsCategories'] = pd.cut(kills['kills'], [-1, 0, 2, 5, 10, 60], labels=['0_kills','1-2_kills', '3-5_kills', '6-10_kills', '10+_kills'])

plt.figure(figsize=(15,8))
sns.boxplot(x="killsCategories", y="winPlacePerc", data=kills)
plt.show()

## The Runners

<img src="https://steemitimages.com/DQmRmYLRxu1vUhVtnFAA6bHFbShtr7Wdv1wLrPjdxbRZsjc/maxresdefault%20(2).jpg" alt="The Runners" style="width: 700px;"/>

In [ ]:
print("The average person walks for {:.1f}m, 99% of people have walked {}m or less, while the marathoner champion walked for {}m.".format(train['walkDistance'].mean(), train['walkDistance'].quantile(0.99), train['walkDistance'].max()))

In [ ]:
data = train.copy()
data = data[data['walkDistance'] < train['walkDistance'].quantile(0.99)]
plt.figure(figsize=(15,10))
plt.title("Walking Distance Distribution",fontsize=15)
sns.distplot(data['walkDistance'])
plt.show()

In [ ]:
print("{} players ({:.4f}%) walked 0 meters. This means that they die before even taking a step or they are afk (more possible).".format(len(data[data['walkDistance'] == 0]), 100*len(data1[data1['walkDistance']==0])/len(train)))

In [ ]:
sns.jointplot(x="winPlacePerc", y="walkDistance",  data=train, height=10, ratio=3, color="lime")
plt.show()

どうやら、歩く距離は、順位と大きな相関があるようです。

## The Drivers

<img src="http://cdn.gamer-network.net/2018/metabomb/pubghowtodrivecarsandbikes.jpg" alt="The Drivers" style="width: 700px;"/>

In [ ]:
print("The average person drives for {:.1f}m, 99% of people have drived {}m or less, while the formula 1 champion drived for {}m.".format(train['rideDistance'].mean(), train['rideDistance'].quantile(0.99), train['rideDistance'].max()))

In [ ]:
data = train.copy()
data = data[data['rideDistance'] < train['rideDistance'].quantile(0.9)]
plt.figure(figsize=(15,10))
plt.title("Ride Distance Distribution",fontsize=15)
sns.distplot(data['rideDistance'])
plt.show()

In [ ]:
print("{} players ({:.4f}%) drived for 0 meters. This means that they don't have a driving licence yet.".format(len(data[data['rideDistance'] == 0]), 100*len(data1[data1['rideDistance']==0])/len(train)))

In [ ]:
sns.jointplot(x="winPlacePerc", y="rideDistance", data=train, height=10, ratio=3, color="y")
plt.show()

運転した距離と順位には小さな相関があります。

乗り物を壊せるのは、私の経験から言って、スキルが高いのではないか、と思います。確かめてみましょう。

In [ ]:
f,ax1 = plt.subplots(figsize =(20,10))
sns.pointplot(x='vehicleDestroys',y='winPlacePerc',data=data,color='#606060',alpha=0.8)
plt.xlabel('Number of Vehicle Destroys',fontsize = 15,color='blue')
plt.ylabel('Win Percentage',fontsize = 15,color='blue')
plt.title('Vehicle Destroys/ Win Ratio',fontsize = 20,color='blue')
plt.grid()
plt.show()

私の経験は正しかったようですね。乗り物を壊せると、勝率をあげてくれるようです！

## The Swimmers

<img src="https://i.ytimg.com/vi/tQxzsE0DijQ/maxresdefault.jpg" alt="The Swimmers" style="width: 700px;"/>

In [ ]:
print("The average person swims for {:.1f}m, 99% of people have swimemd {}m or less, while the olympic champion swimmed for {}m.".format(train['swimDistance'].mean(), train['swimDistance'].quantile(0.99), train['swimDistance'].max()))

In [ ]:
data = train.copy()
data = data[data['swimDistance'] < train['swimDistance'].quantile(0.95)]
plt.figure(figsize=(15,10))
plt.title("Swim Distance Distribution",fontsize=15)
sns.distplot(data['swimDistance'])
plt.show()

ほとんどの人は泳いでいません。そこで、泳いだ距離を4つのグループにビン分割して、最終順位と比べてみましょう。

In [ ]:
swim = train.copy()

swim['swimDistance'] = pd.cut(swim['swimDistance'], [-1, 0, 5, 20, 5286], labels=['0m','1-5m', '6-20m', '20m+'])

plt.figure(figsize=(15,8))
sns.boxplot(x="swimDistance", y="winPlacePerc", data=swim)
plt.show()

どうやら、もしあなたが泳ぐとあなたは上位にいけるようです。PUBGにおいて、今は3つのマップがあります。1つのマップにはそもそも水がありません。そのことを覚えておきましょう。もしかしたら、私はどのマップでプレイヤーたちが戦ったか、を調べるかもしれません。

## The Healers

<img src="https://i.ytimg.com/vi/xfI9XljX51k/maxresdefault.jpg" alt="The Healers" style="width: 700px;"/>

In [ ]:
print("The average person uses {:.1f} heal items, 99% of people use {} or less, while the doctor used {}.".format(train['heals'].mean(), train['heals'].quantile(0.99), train['heals'].max()))
print("The average person uses {:.1f} boost items, 99% of people use {} or less, while the doctor used {}.".format(train['boosts'].mean(), train['boosts'].quantile(0.99), train['boosts'].max()))

In [ ]:
data = train.copy()
data = data[data['heals'] < data['heals'].quantile(0.99)]
data = data[data['boosts'] < data['boosts'].quantile(0.99)]

f,ax1 = plt.subplots(figsize =(20,10))
sns.pointplot(x='heals',y='winPlacePerc',data=data,color='lime',alpha=0.8)
sns.pointplot(x='boosts',y='winPlacePerc',data=data,color='blue',alpha=0.8)
plt.text(4,0.6,'Heals',color='lime',fontsize = 17,style = 'italic')
plt.text(4,0.55,'Boosts',color='blue',fontsize = 17,style = 'italic')
plt.xlabel('Number of heal/boost items',fontsize = 15,color='blue')
plt.ylabel('Win Percentage',fontsize = 15,color='blue')
plt.title('Heals vs Boosts',fontsize = 20,color='blue')
plt.grid()
plt.show()

In [ ]:
sns.jointplot(x="winPlacePerc", y="heals", data=train, height=10, ratio=3, color="lime")
plt.show()

In [ ]:
sns.jointplot(x="winPlacePerc", y="boosts", data=train, height=10, ratio=3, color="blue")
plt.show()

そこで、癒しアイテムやブースティングアイテムの使用は勝率と相関があることがわかります。ブースティングはよりその傾向が強まります。

全てのプロットで、値が0の場合、異常な挙動になりますね。

## Solos, Duos and Squads

ゲームにおいては、3つのモードがあります。1つ目はソロモード、友達と遊ぶデュオモード、3人以上の友達と遊ぶチームモードの3つです。100名のプレイヤーが同じサーバーに入り、デュオモードの場合、50のチームが最大になり、チームモードの場合、最大のチーム数は25になります。

In [ ]:
solos = train[train['numGroups']>50]
duos = train[(train['numGroups']>25) & (train['numGroups']<=50)]
squads = train[train['numGroups']<=25]
print("There are {} ({:.2f}%) solo games, {} ({:.2f}%) duo games and {} ({:.2f}%) squad games.".format(len(solos), 100*len(solos)/len(train), len(duos), 100*len(duos)/len(train), len(squads), 100*len(squads)/len(train),))

In [ ]:
f,ax1 = plt.subplots(figsize =(20,10))
sns.pointplot(x='kills',y='winPlacePerc',data=solos,color='black',alpha=0.8)
sns.pointplot(x='kills',y='winPlacePerc',data=duos,color='#CC0000',alpha=0.8)
sns.pointplot(x='kills',y='winPlacePerc',data=squads,color='#3399FF',alpha=0.8)
plt.text(37,0.6,'Solos',color='black',fontsize = 17,style = 'italic')
plt.text(37,0.55,'Duos',color='#CC0000',fontsize = 17,style = 'italic')
plt.text(37,0.5,'Squads',color='#3399FF',fontsize = 17,style = 'italic')
plt.xlabel('Number of kills',fontsize = 15,color='blue')
plt.ylabel('Win Percentage',fontsize = 15,color='blue')
plt.title('Solo vs Duo vs Squad Kills',fontsize = 20,color='blue')
plt.grid()
plt.show()

ふーむ、とても興味深いですね。ソロとデュオは同じような挙動をしますが、チームモードの場合は、倒した数はあまり関係しなくなるようです。

DBNOsは、攻撃した敵の数のことです。"knock"はデュオやチームモードでしか起きません。なぜなら、チームメイトには攻撃されたプレイヤーを復活させるチャンスがあるからです。そのため、攻撃されたプレイヤーは復活するか死ぬかになります。もし、彼が復活すれば、また攻撃された時に、復活できる時間は短くなります。

アシストも、デュオモードやチームモードでしか起きません。これは一般的には倒す行動に対しての関与を意味するからです。同様に、復活もデュオモードやチームモードでしか起きません。

In [ ]:
f,ax1 = plt.subplots(figsize =(20,10))
sns.pointplot(x='DBNOs',y='winPlacePerc',data=duos,color='#CC0000',alpha=0.8)
sns.pointplot(x='DBNOs',y='winPlacePerc',data=squads,color='#3399FF',alpha=0.8)
sns.pointplot(x='assists',y='winPlacePerc',data=duos,color='#FF6666',alpha=0.8)
sns.pointplot(x='assists',y='winPlacePerc',data=squads,color='#CCE5FF',alpha=0.8)
sns.pointplot(x='revives',y='winPlacePerc',data=duos,color='#660000',alpha=0.8)
sns.pointplot(x='revives',y='winPlacePerc',data=squads,color='#000066',alpha=0.8)
plt.text(14,0.5,'Duos - Assists',color='#FF6666',fontsize = 17,style = 'italic')
plt.text(14,0.45,'Duos - DBNOs',color='#CC0000',fontsize = 17,style = 'italic')
plt.text(14,0.4,'Duos - Revives',color='#660000',fontsize = 17,style = 'italic')
plt.text(14,0.35,'Squads - Assists',color='#CCE5FF',fontsize = 17,style = 'italic')
plt.text(14,0.3,'Squads - DBNOs',color='#3399FF',fontsize = 17,style = 'italic')
plt.text(14,0.25,'Squads - Revives',color='#000066',fontsize = 17,style = 'italic')
plt.xlabel('Number of DBNOs/Assits/Revives',fontsize = 15,color='blue')
plt.ylabel('Win Percentage',fontsize = 15,color='blue')
plt.title('Duo vs Squad DBNOs, Assists, and Revives',fontsize = 20,color='blue')
plt.grid()
plt.show()

## 変数同士の相関係数

In [ ]:
f,ax = plt.subplots(figsize=(15, 15))
sns.heatmap(train.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)
plt.show()

何か目的となる変数について（ここでは勝率）、そこにはいくつかの中くらいから大きく相関する変数が存在します。最も大きく相関するのは、walkDistanceであり、最もネガティブに相関するのはkillPlaceでした。

**今回の目的に最も相関するトップ5について詳細に見ていきましょう**

In [ ]:
k = 5 #number of variables for heatmap
f,ax = plt.subplots(figsize=(11, 11))
cols = train.corr().nlargest(k, 'winPlacePerc')['winPlacePerc'].index
cm = np.corrcoef(train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

## 特徴量エンジニアリング

PUBGのゲームには、最大100名のプレイヤーが互いに戦います。しかし、多くの場合、ゲームは満員というわけではありません。何人のプレイヤーが参加しているか、という変数はないようです。そこで実際に作ってみましょう。

In [ ]:
train['playersJoined'] = train.groupby('matchId')['matchId'].transform('count')

In [ ]:
data = train.copy()
data = data[data['playersJoined']>49]
plt.figure(figsize=(15,10))
sns.countplot(data['playersJoined'])
plt.title("Players Joined",fontsize=15)
plt.show()

"playersJoined"という特徴量は、他のものを正規化することができます。例えば、"killsNorm"や"damageDealtNorm"という特徴量を作ったとします。もし、そこに100名のプレイヤーがいたほうが90名のプレイヤーがいたときよりも、探して倒すのは簡単ではないでしょうか。そこで、私は、正規化するために、100名のプレイヤーの時の倒した数に対しては、1をかけ（そのものの数字）、90名のプレイヤーのときは(100-90)/100 + 1 = 1.1倍をかける、ということができます。これはあくまで仮定ですので、他のスケールを使ってもいいでしょう。

In [ ]:
train['killsNorm'] = train['kills']*((100-train['playersJoined'])/100 + 1)
train['damageDealtNorm'] = train['damageDealt']*((100-train['playersJoined'])/100 + 1)
train[['playersJoined', 'kills', 'killsNorm', 'damageDealt', 'damageDealtNorm']][5:8]

他にの単純な特徴量には、癒しアイテムやブーストアイテムの使用回数の合算があります。もしくは、最終移動距離の合算もあるでしょう。

In [ ]:
train['healsAndBoosts'] = train['heals']+train['boosts']
train['totalDistance'] = train['walkDistance']+train['rideDistance']+train['swimDistance']

もし、ブースティングアイテムを使うと、あなたのスピードは速くなります。PUBG用語で言う「ゾーンから離れる」ことや「もっと多くの略奪（もっと歩くという意味です）」に役立つでしょう。そこで、歩いた距離ごとに使われたブースティングアイテムの量という特徴量を作ってみましょう。癒しアイテムはあなたのスピードを速めませんが、「ゾーンから離れる」ことや「もっと多くの略奪（もっと歩くという意味です）」には役立つでしょう。そこで、癒しアイテムについても同様の特徴量を作ってみましょう。

In [ ]:
train['boostsPerWalkDistance'] = train['boosts']/(train['walkDistance']+1) #The +1 is to avoid infinity, because there are entries where boosts>0 and walkDistance=0. Strange.
train['boostsPerWalkDistance'].fillna(0, inplace=True)
train['healsPerWalkDistance'] = train['heals']/(train['walkDistance']+1) #The +1 is to avoid infinity, because there are entries where heals>0 and walkDistance=0. Strange.
train['healsPerWalkDistance'].fillna(0, inplace=True)
train['healsAndBoostsPerWalkDistance'] = train['healsAndBoosts']/(train['walkDistance']+1) #The +1 is to avoid infinity.
train['healsAndBoostsPerWalkDistance'].fillna(0, inplace=True)
train[['walkDistance', 'boosts', 'boostsPerWalkDistance' ,'heals',  'healsPerWalkDistance', 'healsAndBoosts', 'healsAndBoostsPerWalkDistance']][40:45]

同じですね。では、"killsPerWalkDistance"という特徴量を作ってみましょう。歩いた距離に対しての倒した数の比率ですね。

In [ ]:
train['killsPerWalkDistance'] = train['kills']/(train['walkDistance']+1) #The +1 is to avoid infinity, because there are entries where kills>0 and walkDistance=0. Strange.
train['killsPerWalkDistance'].fillna(0, inplace=True)
train[['kills', 'walkDistance', 'rideDistance', 'killsPerWalkDistance', 'winPlacePerc']].sort_values(by='killsPerWalkDistance').tail(10)

一切歩いていないのに、多くを倒している？しかも、winPlacePerc=1まであります。間違いなく、ズルをしていますね。

一足先にこのカーネルでは、ソロ、デュオ、チームに対するEDAをやってみました。彼らのカラムを作ってみましょう。

In [ ]:
train['team'] = [1 if i>50 else 2 if (i>25 & i<=50) else 4 for i in train['numGroups']]

In [ ]:
train.head()

そこで、私たちは10個の特徴量を作ることができました。これが使いやすいといいな、と思っています！

### ここまで到達してくれてありがとう！これが私の最初のカーネルなんです。もし、上で投票（upvote）してくれたら、とても感謝します。